In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from FeatureSelection import *
from utils import *
%matplotlib inline

In [94]:
# Importing data

test = pd.read_csv('dataset/test.csv')
train = pd.read_csv('dataset/train.csv')

y_feature = 'SalePrice'

# Droping All the columns with more than half of rows nans 
# simply because they might not be as important as the other columns
train.dropna(thresh=train.shape[0] / 2, axis=1, inplace=True)

# Getting the features with missing values
# Divifing the dataset into categorical and numerical
categorical_data = train.select_dtypes('object')
numerical_data = train.select_dtypes(['float64', 'int64'])

# Get the columns with missing data
missing_categorical_data = categorical_data.columns[categorical_data.isna().any()].tolist()
missing_numerical_data = numerical_data.columns[numerical_data.isna().any()].tolist()

# Imputating data with KNN: Try doing this with make pipleline
for feature in missing_numerical_data:
    train[feature] = fillNaWithKNN(feature, train, y_feature)

all_features_list = train.corr()['SalePrice'][1:-1].keys().to_list()
poly_train = polynomial_options(train, all_features_list, 'SalePrice')

handPicked_features = [
    'OverallQual^3', 'OverallQual^4', 'OverallQual^2', 'OverallQual', 
    'GrLivArea', 'GarageCars^2'
]
doubted_features = poly_train.corr()['SalePrice'].nlargest(50)[20:26].keys().to_list()

In [85]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor(eta0=0.55, max_iter=1600, n_iter_no_change=45)
X_train, X_test, y_train, y_test = train_test_split(poly_train[handPicked_features], train[y_feature], test_size=0.33, random_state=42)

sgd.fit(poly_train[handPicked_features], train['SalePrice'])
sgd.fit(X_train, y_train)
r2_score(y_true=y_test, y_pred=sgd.predict(X_test))

-1.9012474697028142e+63

In [86]:
sgd.fit(poly_train[handPicked_features], train[y_feature])

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.55, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1600,
             n_iter_no_change=45, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [87]:
from sklearn.preprocessing import normalize

# Imputating the test data
numerical_test = test.select_dtypes(['float64', 'int64'])
numerical_test.fillna(numerical_test.mean())
X_test = polynomial_options(numerical_test, test.corr()[1:-1], y_feature)
X_test = (X_test - X_test.mean()) / X_test.std()

In [91]:
X_test

,OverallQual^3,OverallQual^4,OverallQual^2,OverallQual,GrLivArea,GarageCars^2,YearBuilt^4
0,-0.753565,-0.705972,-0.775213,-0.750844,-1.215171,-0.963646,-0.365773
1,-0.255868,-0.318894,-0.166772,-0.054858,-0.323429,-0.963646,-0.463968
2,-0.753565,-0.705972,-0.775213,-0.750844,0.294407,0.098867,0.848191
3,-0.255868,-0.318894,-0.166772,-0.054858,0.242921,0.098867,0.882864
4,1.363016,1.296335,1.381986,1.337113,-0.424342,0.098867,0.675606
...,...,...,...,...,...,...,...
1454,-1.087187,-0.918836,-1.273028,-1.446829,-0.811519,-1.317817,-0.068474
1455,-1.087187,-0.918836,-1.273028,-1.446829,-0.811519,-0.963646,-0.068474
1456,-0.753565,-0.705972,-0.775213,-0.750844,-0.539671,0.098867,-0.398555
1457,-0.753565,-0.705972,-0.775213,-0.750844,-1.062772,-1.317817,0.675606


In [89]:
X_test = X_test[handPicked_features]

In [90]:
pred = sgd.predict(X_test)

output = pd.DataFrame({'Id': test.Id,
                      'SalePrice': pred})
output.to_csv('submission.csv', index=False)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').